In [1]:
from sympy import symbols, latex, solve, nsolve
from more_itertools import one
from functools import partial
from IPython.display import display, Math

display_tex = display..Math
real_symbs = symbols$(real=True, positive=True)
def destar(*args) = args
inverse = (/)$(1)
inverse_sum = inverse..sum..map$(inverse)..destar

def solve_disp(*args):
    *lr, var = args
    eq = Eq(*lr) if len(lr) == 2 else one(lr)
    sol = one(solve(eq, var))
    display_tex(
        f"{{{latex(eq)}}}"
        r" \quad \Rightarrow \quad "
        f"{{{latex(Eq(var, sol))}}}")
    return sol

In [2]:
from sympy import Tuple
r, vc, vcc, rint = real_symbs("R, V_{\\mathrm{control}}, V_{cc}, R_{\\mathrm{internal}}")

vc_0 = inverse_sum(2*rint, r)/(rint+inverse_sum(2*rint, r))*vcc
vc_vcc = 2*rint/(2*rint+inverse_sum(rint, r))*vcc

a_0 = vcc/vc_0
a_vcc = vcc/vc_vcc

Tuple(a_0.simplify(), a_vcc.simplify())

(3/2 + R_{\mathrm{internal}}/R, (3*R/2 + R_{\mathrm{internal}})/(R + R_{\mathrm{internal}}))

In [3]:
from sympy import Eq, exp, solve, Rational as R
t_h, t_l, tau, alpha = real_symbs("T_H T_L \\tau \\alpha")
vc_ = vcc/alpha
alpha_ = solve_disp(vc, vc_, alpha)
t_h_eq = Eq(vc, vcc*(1-exp(-t_h/tau))+vc/2*exp(-t_h/tau))
t_h_ = solve_disp(t_h_eq.subs({vc: vc_}), t_h)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [4]:
t_l_eq = Eq(R("1/2"), exp(-t_l/tau))
t_l_ = solve_disp(t_l_eq, t_l)


<IPython.core.display.Math object>

In [5]:
t = real_symbs("T")
t_ = t_h_ + t_l_
alpha_ = solve_disp(t, t_, alpha)

<IPython.core.display.Math object>

In [6]:
tf, ts = real_symbs("T_\\mathrm{fast} T_\\mathrm{slow}")
sys = Tuple(Eq(alpha_.subs({t: ts}), a_vcc.simplify()), Eq(alpha_.subs({t: tf}), a_0.simplify()))
#solve(sys, tau, r)
sys

(Eq((exp(T_\mathrm{slow}/\tau) - 1)/(exp(T_\mathrm{slow}/\tau) - 2), (3*R/2 + R_{\mathrm{internal}})/(R + R_{\mathrm{internal}})), Eq((exp(T_\mathrm{fast}/\tau) - 1)/(exp(T_\mathrm{fast}/\tau) - 2), 3/2 + R_{\mathrm{internal}}/R))

In [7]:
r_ = solve_disp(sys[1], r)

<IPython.core.display.Math object>

This has no closed form solution, we'll need to use a numerical solving method

In [8]:
sys[0].subs({r: r_}).simplify()

Eq(2 - 2*exp(-T_\mathrm{fast}/\tau), (exp(T_\mathrm{slow}/\tau) - 1)/(exp(T_\mathrm{slow}/\tau) - 2))

In [9]:
from sympy.physics.units import Quantity, kg, m, seconds
from sympy import Mul, pi, sqrt, Tuple
from more_itertools import unzip

def split_unit(expr if not expr.has(Quantity)) = (expr, 1)

addpattern def split_unit(expr is Quantity) = (1, expr)

addpattern def split_unit(expr is Mul):
    factors, units = expr.args |> map$(split_unit) |> unzip
    return prod(factors), prod(units) where:
        prod = reduce$(*)

addpattern def split_unit(expr) = (1, expr)

kg*m/sqrt(pi)/seconds**2 |> split_unit |*> Tuple

(1/sqrt(pi), kilogram*meter/second**2)

In [10]:
from sympy import Dict, Expr

def my_nsolve(sys, vars, basis):
    syms, inits = vars.items() |> unzip |> map$(list)
    numerize = map$(basis.to_numeric$(as_expr=True))
    sol = nsolve(numerize(sys), syms, numerize(inits))
    to_unit_of = (quant, numeric) -> basis.to_symb(quant |> split_unit |> .[1], numeric)
    return Dict({sym: to_unit_of(init, sol) for sol, sym, init in zip(sol, syms, inits)})

In [11]:
from sympy import Dict
from utils.units import basis, to_basis
from utils.units.definitions import kHz, kohm, uF, nF, ohm
from more_itertools import unzip

vals = {rint: 5*kohm, ts: 1/(1.8*kHz), tf: 1/(2.3*kHz)}
x0 = {tau: 1/(2.05*kHz), r: (10*kohm)}
(vals, x0) |> map$(Dict) |*> Tuple

({R_{\mathrm{internal}}: 5*kiloohm, T_\mathrm{fast}: 0.434782608695652/kilohertz, T_\mathrm{slow}: 0.555555555555556/kilohertz}, {R: 10*kiloohm, \tau: 0.487804878048781/kilohertz})

In [12]:
sol = my_nsolve(sys.subs(vals), x0, basis())
sol

{R: 17.7819347492499*kiloohm, \tau: 0.366016748307467/kilohertz}

In [13]:
from utils.units import to_basis
from sympy.physics.units.systems import SI
def uconv(quant, target, sys=SI) = to_basis(sys, quant)/to_basis(sys, target) |> .simplify() |> (*)$(target)

Adoptamos un valor de capacitor para obterner el valor de la resistencia que conforma la red RC

In [14]:
from sympy import symbols, Dict, N as approx
from toolz import valmap
r1, c1 = symbols("R_1 C_1")
c1_ = 10*nF
{c1: c1_, r1: uconv(sol[tau]/c1_, kohm), r: sol[r]} |> valmap$(approx$(n=4)) |> Dict

{C_1: 10.0*nanofarad, R: 17.78*kiloohm, R_1: 36.6*kiloohm}